First, setup the environment so that we have graphics

In [1]:
%matplotlib inline
from pylab import rcParams
rcParams['figure.figsize'] = (12, 8)  # Figure size for inline display
import os

Import pynx classes

In [2]:
from pynx.ptycho.runner.id13 import PtychoRunnerScanID13, params
print('Import OK')

gpyfft version: 0.7.0
Using SILX for spec access!
Import OK


First, make sure all results will be saved in **your sub-directory**, so that all results don't collide !

In [3]:
params['saveprefix']='vincent/ResultsScan%04d/Run%04d'

Set the parameters for the optimization.
In this case the scan is fast (50ms/point), so intensity is low - crop the data to 256 pixels to save memory

In [4]:
params['specfile']='../20161021id13-siemens2/siemens2.dat'
params['scan']=14
params['h5file']='../20161021id13-siemens2/pst_fix_he_199_master.h5'
params['probe']='40e-6x40e-6,0.01'  # Starting from a simulated probe, here described as gaussian
params['defocus']=-100e-6        #Defocus the calculated initial probe by 100 microns
params['detectordistance']=1.7598
params['ptychomotors']='nnp2,nnp3,-x,y'   # id13 motors
params['algorithm'] = '20DM'    # Begin with difference map, object only
params['object'] = 'random,0.95,1,0,0.2'   # High energy => start from an almost-flat object
params['verbose'] = 10          # Print every 10 cycles
params['liveplot'] = True       # Liveplot updated at the end of each cell.
#params['moduloframe'] =2,0      # Take only half frames (faster, less memory used)
params['maxsize'] = 256         # Use only 256 pixels (faster, less memory used)

In [5]:
ws = PtychoRunnerScanID13(params, params['scan'])

Load 2000 frames from a 4M Eiger detector (hdf5 format). This takes a little time

In [ ]:
ws.load_data()

Read scan: 14  spiralscan nnp3 nnp2 0.1 2000 0.05
Pixelsize? 7.500000356230885e-05
Reading HDF5 frames: 
Reading h5 data entry: entry/data/data_000001
0 
MASKING 250521 pixels from detector flags
20 40 60 80 100 120 140 160 180 200 
Reading h5 data entry: entry/data/data_000002
220 240 260 280 300 320 340 360 380 400 
Reading h5 data entry: entry/data/data_000003
420 440 460 480 500 520 540 560 580 600 
Reading h5 data entry: entry/data/data_000004
620 640 660 680 700 720 740 760 780 800 
Reading h5 data entry: entry/data/data_000005
820 840 860 880 900 920 940 960 980 1000 
Reading h5 data entry: entry/data/data_000006
1020 1040 1060 1080 1100 1120 1140 1160 1180 1200 
Reading h5 data entry: entry/data/data_000007
1220 1240 1260 1280 1300 1320 1340 1360 1380 1400 
Reading h5 data entry: entry/data/data_000008
1420 1440 1460 1480 1500 1520 1540 1560 1580 1600 
Reading h5 data entry: entry/data/data_000009
1620 1640 1660 1680 1700 1720 1740 1760 1780 1800 
Reading h5 data entry: entry/d

In [ ]:
ws.center_crop_data()  # Auto-crop the data
ws.prepare()  # This will prepare the initial object

Initialized mask with 250521 ( 5.585%) bad pixels


In [ ]:
ws.run()  # Run the optimization,as defined in params['algorithm']

In [ ]:
ws.run_algorithm('probe=1,40DM')  # Activate probe optimization, continue difference map

In [ ]:
ws.run_algorithm('50AP')

In [ ]:
ws.run_algorithm('50ML')

In [ ]:
ws.run_algorithm('Analyze')  # Analyze the optimized probe (modes, propagation)

### Restart using the found probe
Restart with the same probe, and a random object

In [ ]:
params['loadprobe'] = os.path.join(os.path.dirname(params['saveprefix']%(params['scan'], 0)),'latest.npz')
ws.prepare()
ws.run()

In [ ]:
ws.run_algorithm('probe=1,80DM')

In [ ]:
ws.run_algorithm('100AP')

In [ ]:
ws.run_algorithm('100ML')

In [ ]:
ws.run_algorithm('Analyze')  # Analyze the optimized probe (modes, propagation)

In [ ]:
ws.run_algorithm('nbprobe=3,100AP')

In [ ]:
ws.run_algorithm('100ML')

In [ ]:
ws.save_plot(ws._run, display_plot=True) # This makes a more elaborate plot than 'liveplot'

In [ ]:
ws.run_algorithm('Analyze')  # Analyze the optimized probe (modes, propagation)

In [ ]:
ws.plot_llk_history()